In [68]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [69]:
%%sql
drop table if exists tbl;
create table tbl (
    id integer primary key,
    group_id integer not null,
    value integer not null
);

insert into tbl
values (1,1,4),
    (2,1,6),
    (3,1,7),
    (4,1,0),
    (5,2,5),
    (6,2,10),
    (7,3,8);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
7 rows affected.


[]

## やりたいこと
+ 各グループで2番目に大きい要素を取得する

In [74]:
%%sql
-- cross joinでt1は最大、t2はt1をもとに2番目に大きいものに絞り込む
select t1.id,
    t1.group_id,
    t2.value
from tbl as t1, tbl as t2
where t1.group_id = t2.group_id and
t1.value = (
    select max(t3.value)
    from tbl as t3
    where t3.group_id = t1.group_id
) and
t2.value = (
    select max(t3.value)
    from tbl as t3
    where t3.group_id = t2.group_id and
    t3.value < t1.value
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


id,group_id,value
3,1,6
6,2,5


In [78]:
%%sql
-- cross join
select
    max(t1.id),
    t1.group_id,
    max(t1.value)
from tbl as t1, tbl as t2
where t1.group_id = t2.group_id and
t1.value <= t2.value
group by t1.group_id, t1.value
having count(*) = 2

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


max,group_id,max_1
2,1,6
3,1,7
6,2,10
5,2,5
7,3,8


In [21]:
%%sql
-- 相関サブクエリ
select *
from tbl as t1
where value = (
    select max(t2.value)
    from tbl as t2
    where t1.group_id = t2.group_id and
    t2.value < (
        select max(t3.value)
        from tbl as t3
        where t1.group_id = t3.group_id
    )
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


id,group_id,value
2,1,6
5,2,5


In [65]:
%%sql
-- window関数
select *
from (
    select *,
        nth_value(value,2) over (
            partition by group_id
            order by value desc
        ) as second
    from tbl
) as tmp
where value = second

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


id,group_id,value,second
2,1,6,6
5,2,5,5


In [52]:
%%sql
-- 要素が1の場合も候補として出したい時
select min(id), group_id,
    case when count(*) >= 2 then min(value) else NULL end as val
from (
    select t1.id,
        min(t1.group_id) as group_id,
        min(t1.value) as value,
        min(t2.value) as nxt
    from tbl as t1
    left join tbl t2
    on t1.group_id = t2.group_id
    and t1.value < t2.value
    group by t1.id
    having count(*) = 1
) as tmp
group by group_id

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


min,group_id,val
2,1,6
5,2,5
7,3,None
